In [1]:
import pickle
from agents.nnAgent import nnModel, train_test_splitter
from agents.policy_gradient import PolicyModelAgent
from agents.rule_based_agent import RuleBasedAgent
from agents.greedy_agent import GreedyAgent
import tensorflow as tf
from helpers import *

Loading environment football failed: No module named 'gfootball'


In [2]:
def compute_advantage(data, model):
    cur_state_val = model.predict(data['state'])
    next_state_val = np.array(data['reward']).reshape(-1, 1) + 0.85*model.predict(data['next_state']).reshape(-1, 1)*(np.array(data['done']) == False).reshape(-1, 1)
    return (next_state_val - cur_state_val).reshape(-1, 1)

def compute_target_v(data, model):
    next_state_val = np.array(data['reward']).reshape(-1, 1) + 0.85*model.predict(data['next_state']).reshape(-1, 1)*(np.array(data['done']) == False).reshape(-1, 1)
    return next_state_val.tolist()

In [3]:
tf.compat.v1.reset_default_graph()

policy_model = nnModel('policy')
policy_model.build_model(3)

v_model1 = nnModel('state_value')
v_model1.build_model(3)

v_model2 = nnModel('state_value')
v_model2.build_model(3)

actor_critic = nnModel('actor_critic', entropy_weight=0)
actor_critic.build_model(3)

In [4]:
policy_model.load('rule_based_policy')
v_model1.load('state_value')
v_model2.load('state_value')

In [5]:
# Starting point - weights from model trained on predicting rule-based actions
actor_critic.replace_weights(policy_model)

In [6]:
nb_passes = 5
nb_files = 40
nb_epochs_reinforce = 1
nb_epochs_v = 5
batch_size = 32

reload_weights = 5
reload_it = 1

In [7]:
env = make("hungry_geese", debug=True)
config = env.configuration

In [8]:
step_reward = 1
winning_reward = 10
losing_reward = -10
discount = 0.85


def discounted(discount_factor, nb_steps, step_reward):
    discounted_reward = 0
    for _ in range(int(nb_steps)):
        discounted_reward = step_reward + discount_factor*discounted_reward
    return discounted_reward

step_200_reward = lambda my_goose, longuest_opponent: winning_reward if my_goose > longuest_opponent else 3*losing_reward
win_game_reward = lambda step, my_goose, longuest_opponent: winning_reward + discounted(discount, 200-step, step_reward) #max((200-step), winning_reward)

nb_opponents = 3

steps_per_ep = 200
num_episodes = 100
nb_updates = 100

In [ ]:
for it in range(nb_updates):
    print(f'starting update {it}')
    episodes = []
    nb_wins = 0
    for ep in range(num_episodes):
        print('episode number: ', ep)
        steps = []
        my_agent = PolicyModelAgent(actor_critic)
        agents =  [my_agent] + [(RuleBasedAgent() if np.random.rand()<1 else GreedyAgent()) for _ in range(nb_opponents)]
        state_dict = env.reset(num_agents=nb_opponents + 1)[0]
        observation = state_dict['observation']
        my_goose_ind = observation['index']

        reward = state_dict['reward']
        action = state_dict['action']



        done = False
        for step in range(1, steps_per_ep):
            actions = []

            for i, agent in enumerate(agents):
                obs = deepcopy(observation)
                obs['index'] = i
                action = agent(obs, config)
                actions.append(action)

            state_dict = env.step(actions)[0]
            observation = state_dict['observation']
            my_goose_ind = observation['index']

            my_goose_length = len(observation['geese'][my_goose_ind])

            longuest_opponent=0
            for i, goose in enumerate(observation.geese):
                if i != my_goose_ind:
                    opponent_length = len(goose)
                    if opponent_length > longuest_opponent:
                        longuest_opponent = opponent_length

            #new_state, _, _ = agent.getStateSpace(observation, config)

            #reward = state_dict['reward']
            action = state_dict['action']
            status = state_dict['status']

            if status != "ACTIVE":
                done = True

            # Check if my goose died
            if my_goose_length == 0:
                done = True
                reward = losing_reward
            elif (step+1) == steps_per_ep:
                reward = step_200_reward(my_goose_length, longuest_opponent)
                done = True
                if my_goose_length > longuest_opponent:
                    nb_wins += 1
            elif status != "ACTIVE":
                reward = win_game_reward(step, my_goose_length, longuest_opponent)
                nb_wins += 1
            else:
                reward = step_reward

            steps.append({'cur_state': my_agent.stateSpace,
                                    'action': action,
                                    'reward': reward,
                                    'new_state': '',#new_state,
                                    'status': status,
                                    'done': done})
            if done:
    #                 print('Done, Step: ', step+1)
    #                 print('status, ', status)
                break

            if step%50 == 0:
                pass
                #print(f'We survived {step+1} steps')       
        episodes.append(steps)
    print(f'won {100*float(nb_wins)/num_episodes}% games') 
    process(discount, episodes)
    data = training_data(episodes)

    # training models
    X = data['state']
    y = data['y']
    v = compute_target_v(data, v_model2)
    X_train, X_test, y_train, y_test, v_train, v_test = train_test_splitter(X, y, 0.05, v=v)

    advantage = compute_advantage(data, v_model2)

    nb_samples = y.shape[0]
    shuffled = np.random.choice(nb_samples, nb_samples, replace=False)
    new_X = [col[shuffled] for col in X+[np.array(data['v']).reshape(-1,1)]]
    
    actor_critic.fit(new_X,
                     y[shuffled],
                     X_test=None,
                     y_test=None,
                     epoch=nb_epochs_reinforce,
                     batch_size=batch_size)

    actor_critic.save('actor_critic')

    v_model1.fit(X_train,
                 v_train,
                 X_test,
                 v_test,
                 epoch=nb_epochs_v,
                 batch_size=batch_size)

    v_model1.save('v_model_temp_diff')
    if reload_it%reload_weights == 0:
        v_model2.replace_weights(v_model1)
    reload_it+=1

starting update 0
episode number:  0


/home/charles/anaconda3/envs/HungryGeese/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Opposite action: (0, <Action.WEST: 4>, <Action.EAST: 2>)
episode number:  1
Goose Collision: SOUTH
Goose Collision: WEST
episode number:  2
Opposite action: (0, <Action.WEST: 4>, <Action.EAST: 2>)
episode number:  3
Opposite action: (0, <Action.WEST: 4>, <Action.EAST: 2>)
episode number:  4
Opposite action: (0, <Action.SOUTH: 3>, <Action.NORTH: 1>)
episode number:  5
Opposite action: (0, <Action.WEST: 4>, <Action.EAST: 2>)
episode number:  6
Body Hit: (2, <Action.NORTH: 1>, 31, [42, 43, 32, 31, 30, 41, 52])
Goose Collision: SOUTH
Goose Collision: WEST
episode number:  7
Body Hit: (3, <Action.EAST: 2>, 27, [26, 37, 38, 27, 28, 17, 18])
Goose Collision: NORTH
Goose Collision: EAST
episode number:  8
Opposite action: (0, <Action.WEST: 4>, <Action.EAST: 2>)
episode number:  9
Goose Collision: EAST
Goose Collision: WEST
Body Hit: (0, <Action.NORTH: 1>, 74, [8, 9, 10, 0, 1, 67, 66, 55, 65, 64, 75, 74, 73, 7, 18, 19, 20])
episode number:  10
Opposite action: (0, <Action.SOUTH: 3>, <Action.NOR

Opposite action: (0, <Action.SOUTH: 3>, <Action.NORTH: 1>)
episode number:  95
Opposite action: (0, <Action.SOUTH: 3>, <Action.NORTH: 1>)
episode number:  96
Opposite action: (0, <Action.WEST: 4>, <Action.EAST: 2>)
episode number:  97
Goose Collision: NORTH
Goose Collision: EAST
Goose Collision: SOUTH
episode number:  98
Goose Collision: NORTH
Body Hit: (0, <Action.WEST: 4>, 33, [34, 35, 24, 23, 22, 33, 43])
episode number:  99
Goose Collision: EAST
Body Hit: (0, <Action.WEST: 4>, 9, [10, 21, 11, 0, 66, 76, 75, 9, 20, 31, 32, 22, 33])
won 8.0% games
Train on 4826 samples
4826/4826 [==============================] - 7s 1ms/sample - loss: 12.4029
INFO:tensorflow:Assets written to: /home/charles/PycharmProjects/HungryGeese/models/actor_critic/assets
Train on 4584 samples
Epoch 1/5
4584/4584 [==============================] - ETA: 0s - loss: 0.5083 - mae: 0.5083

/home/charles/anaconda3/envs/HungryGeese/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Validation r2: 0.004524498000063826
4584/4584 [==============================] - 7s 1ms/sample - loss: 0.5083 - mae: 0.5083
Epoch 2/5
4584/4584 [==============================] - 4s 903us/sample - loss: 0.4765 - mae: 0.4765
Epoch 3/5
4584/4584 [==============================] - 3s 686us/sample - loss: 0.4747 - mae: 0.4747
Epoch 4/5
4584/4584 [==============================] - 4s 939us/sample - loss: 0.4388 - mae: 0.4388
Epoch 5/5
4584/4584 [==============================] - 4s 875us/sample - loss: 0.4574 - mae: 0.4574
INFO:tensorflow:Assets written to: /home/charles/PycharmProjects/HungryGeese/models/v_model_temp_diff/assets
starting update 1
episode number:  0
Opposite action: (0, <Action.WEST: 4>, <Action.EAST: 2>)
episode number:  1
Opposite action: (0, <Action.EAST: 2>, <Action.WEST: 4>)
episode number:  2
Opposite action: (0, <Action.WEST: 4>, <Action.EAST: 2>)
episode number:  3
Opposite action: (0, <Action.EAST: 2>, <Action.WEST: 4>)
episode number:  4
Opposite action: (0, <Acti

In [6]:
env = make("hungry_geese", debug = True)
my_agent = PolicyModelAgent(policy_model, False)
env.run([my_agent] + ["greedy" for i in range(3)])
env.render(mode="ipython", width=600, height=650)

/home/charles/anaconda3/envs/HungryGeese/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
Body Hit: (1, <Action.EAST: 2>, 26, [25, 14, 15, 26, 37, 38])
Opposite action: (2, <Action.NORTH: 1>, <Action.SOUTH: 3>)
Body Hit: (3, <Action.EAST: 2>, 58, [57, 56, 45, 46, 47, 58, 69, 68, 67, 1, 0, 11, 22, 33, 43, 42, 31, 20])


In [ ]:
        X = data['state']
        y = data['y']
        v = compute_target_v(data, v_model2)
        X_train, X_test, y_train, y_test, v_train, v_test = train_test_splitter(X, y, 0.05, v=v)
        
        advantage = compute_advantage(data, v_model2)
        
        actor_critic.fit(X+[advantage],
                         y,
                         X_test=None,
                         y_test=None,
                         epoch=nb_epochs_reinforce,
                         batch_size=batch_size)

In [23]:
X[0].shape

(7, 1)

In [22]:
advantage.shape

(49, 1)

In [20]:
actor_critic.fit(X+[advantage],
                         y,
                         X_test=None,
                         y_test=None,
                         epoch=nb_epochs_reinforce,
                         batch_size=batch_size)

ValueError: All input arrays (x) should have the same number of samples. Got array shapes: [(7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (7, 1), (49, 1)]

In [29]:
X_train[0].shape

(565, 1)

In [30]:
v_train.shape

(336175, 1)

In [25]:
v_model1.fit(X_train,
                 v_train,
                 X_test,
                 v_test,
                 epoch=nb_epochs_v,
                 batch_size=batch_size)

ValueError: Input arrays should have the same number of samples as target arrays. Found 565 input samples and 336175 target samples.

In [31]:

model = v_model2

In [32]:
cur_state_val = model.predict(data['state'])
next_state_val = np.array(data['reward']).reshape(-1, 1) + 0.85*model.predict(data['next_state'])*(np.array(data['done']) == False)


In [36]:
cur_state_val.shape

(595, 1)

In [ ]:
X+[advantage],
                     y,

In [52]:
nb_samples = y.shape[0]
np.random.choice(nb_samples, nb_samples, replace=False)

In [57]:
np.random.choice(nb_samples, nb_samples, replace=False)

array([ 771,  156,  111, ..., 1271,  837,  908])

In [53]:
nb_samples

1367

In [40]:
model.predict(data['next_state']).shape

(595, 1)

In [38]:
np.array(data['reward']).reshape(-1, 1)

array([[  1.        ],
       [-10.        ],
       [  1.        ],
       [  1.        ],
       [-10.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [-10.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [-10.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1.        ],
       [  1

In [37]:
next_state_val.shape

(595, 595)

In [35]:
(next_state_val - cur_state_val).reshape(-1, 1)

array([[-4.52995300e-04],
       [-5.67018366e+00],
       [-4.52995300e-04],
       ...,
       [-1.10013137e+01],
       [-1.10013137e+01],
       [-1.66754260e+01]])

In [10]:
advantage

array([[-1.12581253e-03],
       [-1.66711264e+01],
       [-8.34941864e-04],
       [-4.52518463e-04],
       [-1.10054016e-03],
       [-5.75065613e-04],
       [-2.88486481e-04],
       [-8.09669495e-04],
       [-1.31607056e-04],
       [-1.32465363e-03],
       [-2.50816345e-04],
       [-1.66703973e+01],
       [-1.68895721e-03],
       [-1.66706209e+01],
       [-7.76767731e-04],
       [-1.66712618e+01],
       [-9.51290131e-04],
       [-7.82489777e-04],
       [-9.12189484e-04],
       [ 7.67707825e-05],
       [-1.66706848e+01],
       [-1.13725662e-03],
       [ 1.19352341e-03],
       [-1.50966644e-03],
       [ 4.33921814e-05],
       [-1.06239319e-03],
       [-1.22070312e-04],
       [-9.54627991e-04],
       [ 3.48091125e-05],
       [-1.66710901e+01],
       [-1.65510178e-03],
       [ 1.76429749e-05],
       [-8.84532928e-04],
       [-7.82012939e-05],
       [-4.76837158e-04],
       [-1.31893158e-03],
       [-1.58309937e-04],
       [-1.66712770e+01],
       [-1.9